In [1]:
import numpy as np
from numpy import dot
from sympy import *
import matplotlib.pyplot as plt
import time

def Speedest(f,x,epsilon):
    '''
    最速下降法
    :param f: 函数
    :param x: 初始点
    :param epsilon: epsilon
    :return: [迭代次数，计算时间，最优解，最小值]
    '''
    start=time.time()
    cnt = 0  # 计数标志
    xk = x
    gk = grad(f, xk)
    while (np.linalg.norm(gk) > epsilon):
        dk = -grad(f, xk)
        alpha = Armijo(f, xk, dk, 0.5)  # Armijo求步长
        xk = xk + alpha * dk  # 迭代
        gk = grad(f, xk)  # 迭代后点处的梯度
        cnt = cnt + 1  # 迭代次数+1
        if(cnt==100000):break
    end = time.time()
    print('最速下降法:')
    print('迭代次数:%d'% cnt)
    print('运行时间:'+(str)(end-start)+'s')
    print('最优解为:\n'+(str)(xk))
    print('最小值:%.8f'% f(xk))
    return True

def Newton(f,x,epsilon):
    '''
    牛顿法
    :param f: 函数
    :param x: 初始点
    :param epsilon: epsilon
    :return: [迭代次数，计算时间，最优解，最小值]
    '''
    start = time.time()
    cnt = 0  # 计数标志
    xk = x
    gk = grad(f, xk)  # 梯度
    while (np.linalg.norm(gk) > epsilon):  # 迭代条件:梯度的范数大于epsilon
        if (np.linalg.det(Hessian(f, xk)) < 0.00001):
            dk = -gk
        else:
            dk = -(np.linalg.inv(Hessian(f, xk)).dot(gk))  # 求方向
        xk = xk + dk  # 迭代
        gk = grad(f, xk)  # 迭代后点处的梯度
        cnt = cnt + 1  # 迭代次数+1
        if (cnt == 100000): break
    end = time.time()
    print('牛顿法:')
    print('迭代次数:%d' % cnt)
    print('运行时间:' + (str)(end - start) + 's')
    print('最优解为:\n' + (str)(xk))
    print('最小值:%.8f' % f(xk))
    return True


def DampingNewton(f,x,epsilon):
    '''
        阻尼牛顿法
        :param f: 函数
        :param x: 初始点
        :param epsilon: epsilon
        :return: [迭代次数，计算时间，最优解，最小值]
        '''
    start=time.time()
    cnt=0       #计数标志
    xk=x
    gk=grad(f,xk)       #梯度
    while(np.linalg.norm(gk)>epsilon): #迭代条件:梯度的范数大于epsilon
        if(np.linalg.det(Hessian(f,xk))<0.0001):
            dk=-gk
        else:
            dk=-(np.linalg.inv(Hessian(f,xk)).dot(gk))    #求方向
        #alpha=Armijo(f,xk,dk,0.5)                       #求步长
        alpha=StepSize(f,xk,dk)
        xk=xk+alpha*dk                                  #迭代
        gk=grad(f,xk)                               #迭代后点处的梯度
        cnt=cnt+1                                       #迭代次数+1
        if(cnt==100000): break
    end=time.time()
    print('阻尼牛顿法:')
    print('迭代次数:%d' % cnt)
    print('运行时间:' + (str)(end - start) + 's')
    print('最优解为:\n' + (str)(xk))
    print('最小值:%.8f' % f(xk))
    return True

def DFP(f,xk,epsilon):
    '''
    DFP法
    :param f:函数
    :param xk:初始点
    :param epsilon:epsilon
    :return: [迭代次数，计算时间，最优解，最小值]
    '''
    start=time.time()
    cnt=0
    n=np.size(xk)
    H=np.mat(np.eye(n))
    while(mo(grad(f,xk))>epsilon):
        cnt+=1
        dk=-H.dot(grad(f,xk))
        alpha=Armijo(f,xk,dk,0.5)
        sk=alpha*dk
        yk=grad(f,xk+alpha*dk)-grad(f,xk)
        if(mo(sk.T.dot(yk))==0 or mo(yk.T.dot(H))==0):
            H=np.eye(n)
        else:
            H=H+(sk.dot(sk.T))/mo(sk.T.dot(yk))-(H.dot(yk).dot(yk.T).dot(H))/mo(yk.T.dot(H).dot(yk))
        xk = xk + alpha * dk
        if (cnt == 100000): break
    end=time.time()
    print('拟牛顿法:')
    print('迭代次数:%d' % cnt)
    print('运行时间:' + (str)(end - start) + 's')
    print('最优解为:\n' + (str)(xk))
    print('最小值:%.8f' % f(xk))
    return True

def FR(f,x,epsilon):
    '''
    FR共轭梯度法
    :param f: 函数
    :param x: 初始迭代点x0
    :param epsilon: epsilon
    :return: [迭代次数，计算时间，最优解，最小值]
    '''
    start=time.time()
    cnt=0 #计数标志
    xk=x
    g_pre=grad(f,xk)   #xk-1点的梯度,第一次迭代时，梯度任意
    g_now=grad(f,xk)   #xk点的梯度
    while(mo(g_now)>epsilon):
        cnt+=1
        if(cnt==1):
            dk=-g_now
        else:
            beta = ((g_now.T.dot(g_now)) / (g_pre.T.dot(g_pre)))[0,0]
            dk=-g_now+beta*g_pre
        alpha=Armijo(f,xk,dk,0.5)
        xk=xk+alpha*dk          #计算xk+1
        g_pre=g_now       #对gk-1进行迭代
        g_now=grad(f,xk)     #对gk进行迭代
        if(cnt==100000):break
    end=time.time()
    print('FR共轭梯度法:')
    print('迭代次数:%d' % cnt)
    print('运行时间:' + (str)(end - start) + 's')
    print('最优解为:\n' + (str)(xk))
    print('最小值:%.8f' % f(xk))
    return True

def LMF(f,x,epsilon):
    '''
    LMF方法
    :param f: 函数
    :param x: 初始点
    :param epsilon: eosilon
    :return: [迭代次数，计算时间，最优解，最小值]
    '''
    start=time.time()
    cnt=0                           #计数标志
    xk=x
    vk=1
    gk=grad(f,xk)
    while(mo(gk)>epsilon):
        cnt += 1
        J = (np.mat)(np.zeros([11, 4]))
        rk= (np.mat)(np.zeros([11,1]))
        for i in range(0,11):                   #计算Jaccobi行列式
            r=lambda x:y[i,0]-f_bar(x,t[i,0])       #定义ri(x)
            gr=grad(r,xk)
            for j in range(0,4):
                J[i,j]=gr[j,0]
            rk[i,0]=r(xk)
        I=(np.mat)(np.eye(4))
        J = (np.float64)(J)
        I = (np.float64)(I)
        rk= (np.float64)(rk)
        if(abs(np.linalg.det(J.T.dot(J)+vk*I))<0.1):
            dk=-gk
        else:
            dk=np.linalg.inv(J.T.dot(J)+vk*I).dot(-J.T.dot(rk))
        f_delta=f(xk)-f(xk+dk)
        q_delta=0.5*(dk.T.dot(vk*dk-gk))[0,0]
        gama=f_delta/q_delta
        if(gama<0.25):
            vk=4*vk
        elif (gama>0.75):
            vk=0.5*vk
        if(gama>0):
            xk=xk+dk
            gk=grad(f,xk)
        if(cnt==100000):break
    end=time.time()
    print('LMF法:')
    print('迭代次数:%d' % cnt)
    print('运行时间:' + (str)(end - start) + 's')
    print('最优解为:\n' + (str)(xk))
    print('最小值:%.8f' % f(xk))
    return True

def mo(xk):      #求模
    return np.linalg.norm(xk)

def grad(f, x):
    '''
    求梯度
    :param f: 函数
    :param x: 向量
    :return: 梯度向量
    '''
    delta=0.0000001
    gradmatrix=np.zeros(x.shape)
    fx=f(x)
    it=np.nditer(x,flags=['multi_index'],op_flags=['readwrite'])
    while not it.finished:
        ix=it.multi_index
        old_value=(np.float64)(x[ix])
        x[ix]=(np.float64)(old_value+delta)
        fxd=f(x)
        gradmatrix[ix]=(fxd-fx)/delta
        x[ix]=old_value
        it.iternext()
    return gradmatrix

def Hessian(f,x):
    '''
    求Hessian矩阵
    :param f: 函数
    :param x: 初始点
    :param epsilon: epsilon
    :return: f在x处的Hessian矩阵
    '''
    delta=0.001
    n=np.size(x)
    HessianMatrix=np.zeros((n,n))
    gx0=grad(f,x)
    for i in range(0,n):
        old_value = (np.float64)(x[i,0])
        x[i,0] = (np.float64)(old_value + delta)
        gxk=grad(f,x)
        for j in range(0,n):
            HessianMatrix[i,j]=(np.float64)((gxk[j,0]-gx0[j,0])/delta)
        x[i,0]=old_value
    return HessianMatrix

def Armijo(f,xk,dk,rho):
    '''
    用Armijo算法求步长
    f:函数
    xk:函数原始点
    dk：原始方向向量
    rho:rho
    '''
    cnt=0
    alpha=1                             #初始化alpha
    vtr=xk
    vtr_alpha=xk+alpha*dk
    #while(f(vtr_alpha)>f(vtr)+rho*alpha*grad(f,xk).T.dot(dk) or f(vtr_alpha)<f(vtr)+(1-rho)*alpha*grad(f,xk).T.dot(dk)): #所需满足的条件
    while (f(vtr_alpha) > f(vtr) + rho * alpha * grad(f, xk).T.dot(dk) ):  # 所需满足的条件
        alpha=rho*alpha
        vtr_alpha = xk + alpha * dk
    return alpha

def StepSize(f,xk,dk):
    '''
    二元函数一维精确线搜索求步长
    :param f: 二元函数
    :param xk: 函数点
    :param dk: 方向
    :return: 步长值
    '''
    if(dot(dot(dk.T,Hessian(f,xk)),dk)[0,0]==0):  #分母为0的情况给定步长为0.01
        return 0.01
    else:
        alpha=-(grad(f,xk).T.dot(dk))/(dot(dot(dk.T,Hessian(f,xk)),dk))
        return alpha[0,0]

def f(x):
    x1=x[0,0]
    x2=x[1,0]
    x3=x[2,0]
    x4=x[3,0]
    result=0
    for i in range(0,11):
        result+=(y[i,0]-(x1*(t[i,0]**2+x2*t[i,0])/(t[i,0]**2+x3*t[i,0]+x4)))**2
    return result

def f_bar(x,t):
    x1 = x[0, 0]
    x2 = x[1, 0]
    x3 = x[2, 0]
    x4 = x[3, 0]
    result=x1*(t**2+x2*t)/(t**2+x3*t+x4)
    return result

def g(x):
    x1 = x[0, 0]
    x2 = x[1, 0]
    x3 = x[2, 0]
    x4 = x[3, 0]
    return 3*x1**3+2*x2**2+3*x3+x4*x3**2

y = np.mat([[0.1957], [0.1947], [0.1735], [0.1600], [0.0844], [0.0627],
            [0.0456], [0.0342], [0.0323], [0.0235], [0.0246]])
t = np.mat([[4.0000], [2.0000], [1.0000], [0.5000], [0.2500], [0.1670],
            [0.1250], [0.1000], [0.0833], [0.0714], [0.0625]])
xk=np.mat([[0.0],[0.0],[0.0],[0.0]])
print(f(xk))
print(grad(f,xk))
print(Hessian(f,xk))
Speedest(f,xk,0.0001)
DampingNewton(f,xk,0.0001)
Newton(f,xk,0.0001)
DFP(f,xk,0.0001)
FR(f,xk,0.0001)
LMF(f,xk,0.0001)

0.14841318
[[-2.0623989]
 [ 0.       ]
 [ 0.       ]
 [ 0.       ]]
[[21.99999988 -6.19272467  6.19271967 51.11162465]
 [-6.34020697  0.          0.          0.        ]
 [ 6.28805064  0.          0.          0.        ]
 [46.31880518  0.          0.          0.        ]]
最速下降法:
迭代次数:820
运行时间:1.0485796928405762s
最优解为:
[[0.19385489]
 [0.16561179]
 [0.11663432]
 [0.12450401]]
最小值:0.00030886
阻尼牛顿法:
迭代次数:122
运行时间:0.4381828308105469s
最优解为:
[[0.19143365]
 [0.22210067]
 [0.12931595]
 [0.14991261]]
最小值:0.00030907
牛顿法:
迭代次数:53
运行时间:0.07496261596679688s
最优解为:
[[-1.05047168e+00]
 [-6.12759067e+02]
 [ 6.14638827e+02]
 [ 6.84509712e+03]]
最小值:0.03737007
拟牛顿法:
迭代次数:4846
运行时间:5.650827407836914s
最优解为:
[[0.19297973]
 [0.18705423]
 [0.12170702]
 [0.13431   ]]
最小值:0.00030754
FR共轭梯度法:
迭代次数:100000
运行时间:56.86339712142944s
最优解为:
[[0.19301836]
 [0.0030953 ]
 [0.00556666]
 [0.05702301]]
最小值:0.00073382
LMF法:
迭代次数:444
运行时间:0.3400115966796875s
最优解为:
[[0.19385556]
 [0.16570258]
 [0.11668666]
 [0.12454144]]
最小值:0.00

True